# 基于langchain的大模型检索增强

- 基于langchain 、Chroma 、 LLM
- 步骤：
    - 文档预处理
    - 文档分段
    - 向量化&持久化
    - 向量检索相似度
    - 通过prompt工程对检索结果进行知识增强（注： 大模型也可以API的方式对接）

# 文档预处理
支持 txt \ docx \ md \ pdf

In [1]:
from langchain.document_loaders import UnstructuredFileIOLoader, UnstructuredFileLoader  
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = UnstructuredFileLoader('./inputs/rag.txt')
data = loader.load()
print(data)

[Document(page_content='RAG（Retrieval\n\nAugmented Generation）是一种自然语言处理领域的模型框架，旨在通过将检索和生成两个关键任务相结合，提高文本生成任务的效果。RAG的设计旨在解决生成模型的一个关键问题，即生成过程中缺乏对先前信息的深刻理解和合理利用。通过引入检索机制，RAG使得模型能够从大量的外部知识中检索相关信息，以提高生成结果的质量和准确性。\n\nRAG的核心思想是将一个强大的检索系统嵌入到生成模型中，以便在生成文本的过程中引入外部知识。这一设计灵感源自于生成模型在处理复杂问题时可能面临的信息不足问题。传统的生成模型，如GPT（Generative Pre\n\ntrained Transformer），通常通过大规模无监督训练从大量文本中学习语言模型。尽管这些模型在很多任务上表现出色，但它们在需要特定领域或主题的深入理解时，可能受到数据限制而表现不佳。\n\nRAG的架构由两个主要组件组成：检索器（Retriever）和生成器（Generator）。这两个组件协同工作，以实现更全面的文本生成。下面对这两个组件进行详细介绍：\n\n检索器（Retriever）：\n\nRAG中的检索器负责从大规模的知识库中检索相关信息。这个知识库可以是预先构建的、包含了领域特定知识的文本数据库，也可以是互联网上的开放域知识。\n\n检索器通常使用一些技术，如倒排索引（inverted index）或者基于向量的相似度计算，来高效地从知识库中提取与当前生成任务相关的信息。\n\n检索器的输出是一组文本片段，这些文本片段被认为是与生成任务相关的重要信息。\n\n生成器（Generator）：\n\n生成器是负责实际文本生成的部分，通常是一个预训练的生成模型，比如GPT。生成器使用检索器提供的信息来辅助生成更具准确性和相关性的文本。\n\n生成器在生成文本的过程中可以选择性地引入检索到的知识，以更好地适应特定的上下文和任务要求。\n\n生成器还可以学习在何时、何地以及如何利用检索到的信息，从而实现更精细的文本生成控制。\n\nRAG的工作流程通常包括以下步骤：\n\n检索阶段：输入一个查询，检索器从知识库中检索相关信息。\n\n融合阶段：生成器将检索到的信息与当前上下文融合，形成一个更全面的输入表示。\n\n生成阶段：

# 文本分块
- chunk_size: 每个分片的最大大小、chunk_overlap分片之间的覆盖大小，可以保持连贯性

In [2]:
texp_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)  # 100 characters per chunk with 0 overlap 中文： 100个字，不重叠
split_docs = texp_splitter.split_documents(data)
print(split_docs)

[Document(page_content='RAG（Retrieval\n\nAugmented Generation）是一种自然语言处理领域的模型框架，旨在通过将检索和生成两个关键任务相结合，提高文本生成任务的效果。RAG的设计旨在解决生成模型的一个关键问题，即生成过程中缺乏对先前信息的深刻理解和合理利用。通过引入检索机制，RAG使得模型能够从大量的外部知识中检索相关信息，以提高生成结果的质量和准确性。', metadata={'source': './inputs/rag.txt'}), Document(page_content='RAG的核心思想是将一个强大的检索系统嵌入到生成模型中，以便在生成文本的过程中引入外部知识。这一设计灵感源自于生成模型在处理复杂问题时可能面临的信息不足问题。传统的生成模型，如GPT（Generative Pre', metadata={'source': './inputs/rag.txt'}), Document(page_content='trained Transformer），通常通过大规模无监督训练从大量文本中学习语言模型。尽管这些模型在很多任务上表现出色，但它们在需要特定领域或主题的深入理解时，可能受到数据限制而表现不佳。\n\nRAG的架构由两个主要组件组成：检索器（Retriever）和生成器（Generator）。这两个组件协同工作，以实现更全面的文本生成。下面对这两个组件进行详细介绍：', metadata={'source': './inputs/rag.txt'}), Document(page_content='检索器（Retriever）：\n\nRAG中的检索器负责从大规模的知识库中检索相关信息。这个知识库可以是预先构建的、包含了领域特定知识的文本数据库，也可以是互联网上的开放域知识。\n\n检索器通常使用一些技术，如倒排索引（inverted index）或者基于向量的相似度计算，来高效地从知识库中提取与当前生成任务相关的信息。', metadata={'source': './inputs/rag.txt'}), Document(page_content='检索器的输出是一组文本片段，这些文本片段被认为是与生成任务相关的重要信息。\n\n生成器（Generator）：\n\n生成器是负责实际

# 文本分块转向量并持久化

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import sentence_transformers
embedding_model_dict = {
    "bce-embedding-base_v1":"D:\\JHQ\\modelspace\\bce-embedding-base_v1"
}

EMBEDDING_MODEL = "bce-embedding-base_v1"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_dict[EMBEDDING_MODEL])

# 持久化

In [ ]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(split_docs, embeddings,persist_directory='./persist') 

In [ ]:
db.persist()  # save the database to disk

# 向量检索

In [ ]:
query = "RAG的核心思想是什么"
db = Chroma(persist_directory = './persist',embedding_function=embeddings)  # load the database from disk

In [ ]:
similarDocs = db.similarity_search(query, k=3)

In [ ]:
info = ""
for doc in similarDocs:
    print("----")
    if doc.page_content in info:
        continue
    print(doc.page_content)
    info = info + doc.page_content

# 增强生成

In [ ]:
from openai import OpenAI
# 